In [18]:
import Pkg

Pkg.add(url="https://github.com/matthiasbaitsch/mmjmesh.git");

using IntervalSets
using MMJMesh.Plots
using MMJMesh.Meshes
using MMJMesh.Utilities
using MMJMesh.Mathematics
using LinearAlgebra
using Symbolics
using DomainSets
using CairoMakie

using GLMakie
using WGLMakie
WGLMakie.activate!()
CairoMakie.activate!()

    Updating git-repo `https://github.com/matthiasbaitsch/mmjmesh.git`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [19]:
@variables  a,b,h,ν ;

In [20]:
# a = 4.5
# b = 4.5

V = [-b/2 -b/2 b/2 b/2; -a/2 a/2 a/2 -a/2]
P = mmonomials(2, 3, (-b/2 .. b/2) × (-a/2 .. a/2), (p1, p2) -> p1 + p2 <= 4 && p1 * p2 <4)
N = vcat(
    [
        [
            ValueAtLF(p),
            PDerivativeAtLF(p, [1, 0]),
            PDerivativeAtLF(p, [0, 1])
        ]
        for p in eachcol(V)
    ]...
)
M = [n(p) for p in P, n in N]
Minv = round.(inv(M); digits = 3)
H4 = Minv*P;
#fplot3d(H4)

TypeError: TypeError: in Type, in parameter, expected Type, got a value of type Rectangle{StaticArraysCore.SVector{2, Num}}

In [21]:
L = [1 ν 0; ν 1 0; 0 0 (1-ν)/2]

3×3 Matrix{Num}:
 1  ν               0
 ν  1               0
 0  0  (1//2)*(1 - ν)

In [22]:
ν = 0.5
h = 1
∂1(u) = (1 / a^2) * ∂x(∂x(u))
∂2(u) = (1 / b^2) * ∂y(∂y(u))
∂3(u) = (2 / a*b) * ∂x(∂y(u))
Be1(u) = [∂1(u), ∂2(u), ∂3(u)] 
Be2(u) = [∂1(u) + ν * ∂2(u), ν * ∂1(u) + ∂2(u), 1/2 * (1- ν) * ∂3(u)] 

ae(u,v) = integrate(Be1(u) ⋅ Be2(v), (-b/2 .. b/2) × (-a/2 .. a/2))
Ke = h^3/12 * simplify.(expand.([ae(n1, n2) for n1 ∈ H4, n2 ∈ H4]))

12×12 Matrix{Num}:
 (0.0833333(0.000363(a^6) + 0.000363(b^6) + 6.25e-6(a^6)*(b^2) + 6.25e-6(a^4)*(b^4) + 0.00980725(a^2)*(b^6) - 0.0002475(a^4)*(b^6) - 0.0002475(a^2)*(b^8) + 2.8125e-6(a^6)*(b^6) + 3.125e-6(a^4)*(b^8) + 2.8125e-6(a^2)*(b^10))) / ((a^3)*(b^3))   …  (0.0833333(0.000825(a^4) - 0.0001375(b^4) - 1.375e-5(a^4)*(b^2) - 6.875e-6(a^2)*(b^4) - 0.005544(b^6) + 0.00034225(a^2)*(b^6) + 7.0e-5(b^8) - 6.1875e-6(a^4)*(b^6) - 3.4375e-6(a^2)*(b^8))) / (a*(b^3))
 (0.0833333(0.0198(a^4) + 0.1188(b^4) + 0.00099(a^4)*(b^2) + 0.800316(a^2)*(b^4) - 0.01008(a^4)*(b^4) - 0.049284(a^2)*(b^6) + 0.000495(a^4)*(b^6) + 0.000891(a^2)*(b^8))) / (144(a^3)*b)                                                              (0.0833333(0.006272 + 0.0003125(a^2) - 0.0003125(b^2) - 1.5125e-5(a^2)*(b^2) - 0.003136(b^4) + 0.000154(a^2)*(b^4) + 0.000154(b^6) - 7.5625e-6(a^2)*(b^6))) / (a*b)
 (0.0833333(0.000825(a^4) + 0.0001375(b^4) + 1.375e-5(a^4)*(b^2) + 6.875e-6(a^2)*(b^4) + 0.005544(b^6) - 0.00034225(a^2)*(b^6